In [2]:
import numpy as np
import os
from collections import defaultdict
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from pathlib import Path
import ast
import sys

sys.path.append("..")
import representations.cka_lib as cka_lib

In [3]:
def compute_cka_distance(model_name, df, error, encoding_path, src):
    test_encodings = np.load(os.path.join(encoding_path, f'{model_name}/last/test.{error}.{src}.encoding.npy'), allow_pickle=True).transpose((1, 0))
    clean_test_encodings = np.load(os.path.join(encoding_path, f'{model_name}/last/test.{error}.{src}.clean.encoding.npy'), allow_pickle=True).transpose((1, 0))
    
    word_encodings = [[layer_test_encodings[i][int(row['index'])] for i, row in df.iterrows() if row['label'] not in ['clean']] for layer_test_encodings in test_encodings]
    clean_word_encodings = [[layer_clean_test_encodings[i][int(row['index'])] for i, row in df.iterrows() if row['label'] not in ['clean']] for layer_clean_test_encodings in clean_test_encodings]

    word_encodings = np.array(word_encodings)
    clean_word_encodings = np.array(clean_word_encodings)
    
    layer_cosine_sim = [1 - cka_lib.cka(word_encodings[i], clean_word_encodings[i]).item() for i in range(0, test_encodings.shape[0])]
    return layer_cosine_sim

In [4]:
def compute_cka_distance_multi(model_name, df, error, encoding_path, src):
    test_encodings = np.load(os.path.join(encoding_path, f'{model_name}/last/test.{error}.{src}.encoding.npy'), allow_pickle=True).transpose((1, 0))
    clean_test_encodings = np.load(os.path.join(encoding_path, f'{model_name}/last/test.{error}.{src}.clean.encoding.npy'), allow_pickle=True).transpose((1, 0))
    
    word_encodings = [[layer_test_encodings[i][int(j)] for i, row in df.iterrows() if row['label']!='clean' for j in row['index']] for layer_test_encodings in test_encodings]
    clean_word_encodings = [[layer_clean_test_encodings[i][j] for i, row in df.iterrows() if row['label']!='clean' for j in row['index']] for layer_clean_test_encodings in clean_test_encodings]

    word_encodings = np.array(word_encodings)
    clean_word_encodings = np.array(clean_word_encodings)
    
    layer_cosine_sim = [1 - cka_lib.cka(word_encodings[i], clean_word_encodings[i]).item() for i in range(0, test_encodings.shape[0])]
    return layer_cosine_sim

In [ ]:
# Function to create a similarity DataFrame for different models and errors
def create_sim_df(model_prefix, direction, error_types, encoding_path, base_model_dict, src):
    sim_df = defaultdict(list)
    
    for error in error_types:
        finetuned_model_name = f'{model_prefix}-{direction}-{error}'
        clean_finetuned_model_name = f'{model_prefix}-{direction}-clean-{error}'
        base_model_name = base_model_dict[model_prefix]
        
        if 'opus' in base_model_name:
            base_model_name = f'{base_model_name}-{direction}'
        
        error_path = model_prefix+".morpheus" if error=='morpheus' else error
        df = pd.read_pickle(f'../data/grammar-noise/{direction}/test.{error_path}.{src}.pkl')
        
        for model_type, model_name in [('Noise-Finetuned', finetuned_model_name), ('Clean-Finetuned', clean_finetuned_model_name), ('Base', base_model_name)]:
            encodings = compute_cka_distance_multi(model_name, df, error_path, encoding_path, src)
            sim_df['Model'].append(model_type)
            sim_df['Error'].append(error)
            sim_df['Embedding Distance'].append(encodings)
            sim_df['Model Prefix'].append(model_prefix)
            sim_df['Layer'].append([i for i in range(1, len(encodings) + 1)])
            
    return pd.DataFrame(sim_df)

In [ ]:
# Set up plot configurations
plt.rcParams['figure.dpi'] = 300
sns.set_theme(style='whitegrid', font_scale=1.4)

models = {
    'opus': 'opus-mt',
    'm2m100': 'm2m100_418M', 
    'mbart': 'mbart-large-50-many-to-many-mmt', 
    'nllb': 'nllb-200-distilled-600M', 
}
# error_types = ['article', 'nounnum', 'prep']
error_types = ['morpheus']
figure_path = '../results/figures/CKA_Distance'
distance_path = '../outputs/representation_distance'

combined_df_list = []
for direction in ['en-es', 'fr-es', 'en-de', 'en-it', 'en-nl']:
    encoding_path = f'../outputs/representations/encodings/{direction}'
    src = direction.split('-')[0]
    
    Path(figure_path).mkdir(exist_ok=True, parents=True)
    Path(distance_path).mkdir(exist_ok=True, parents=True)
    
    for model_prefix, model_name in models.items():
        
        # Avoid recomputing distances if they already exist
        distance_file = f'{distance_path}/{model_prefix}_{direction}_distance.csv'
        
        if not os.path.exists(distance_file):
            sim_df = create_sim_df(model_prefix, direction, error_types, encoding_path, models, src)
            sim_df.to_csv(distance_file, index=False)
        else:
            sim_df = pd.read_csv(distance_file)
            sim_df['Embedding Distance'] = sim_df['Embedding Distance'].apply(ast.literal_eval)
            sim_df['Layer'] = sim_df['Layer'].apply(ast.literal_eval)
        
        sim_df = sim_df.explode(['Embedding Distance', 'Layer'])
        sim_df['Language'] = '-'.join([d.capitalize() for d in direction.split('-')])
        combined_df_list.append(sim_df)
        
combined_df = pd.concat(combined_df_list)
combined_df.loc[combined_df['Model Prefix'] == 'opus', 'Model Prefix'] = 'OPUS-MT'
combined_df['Model Prefix'] = combined_df['Model Prefix'].apply(lambda x: x.upper())
combined_df['Error'] = combined_df['Error'].apply(lambda x: x.capitalize())
    
g = sns.catplot(
    x='Layer', 
    y='Embedding Distance', 
    hue='Model', 
    row='Model Prefix', 
    col='Language',        
    data=combined_df, 
    kind='point', 
    height=5, 
    aspect=1.2,
    palette='Set2',
    sharey=True 
)

for ax in g.axes.flat:
    ax.tick_params(axis='x', labelsize=20)
    ax.tick_params(axis='y', labelsize=20)

sns.move_legend(
    g, "lower center",
    bbox_to_anchor=(0.5, -0.03), ncol=3, title=None, frameon=False,
    )

plt.ylim(0.0, 1.0)
plt.subplots_adjust(hspace=0.2, wspace=0.1)

plt.setp(g._legend.get_texts(), fontsize=30)
g.fig.suptitle(f"CKA Distance: Morpheus", fontsize=36, y=1.05)
g.set_axis_labels("Layer", "CKA Distance", fontsize=30)
g.set_titles(col_template="{col_name}", row_template="{row_name}", size=30)

plt.savefig(f'{figure_path}/CKA_distance_morpheus.png', dpi=300, bbox_inches='tight')
plt.savefig(f'{figure_path}/CKA_distance_morpheus.pdf', dpi=300, bbox_inches='tight')

plt.show()
